# Compréhension du problème :
On travaille sur un problème de régression, car on essaye de prédire une valeur numérique continue 'SiteEnergyUse(kBtu)', 'TotalGHGEmissions' à partir de caractéristiques de bâtiments.

# Import

In [15]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score, KFold

# Data

In [2]:
data = pd.read_csv('data_saved.csv')

# Chargement de la liste des targets et features

In [3]:
features = np.load('features.npy')
targets = np.load('targets.npy')

In [4]:
print(features)

['NumberofBuildings' 'PropertyGFATotal' 'PropertyGFABuilding(s)'
 'BuildingAge' 'LargestPropertyUseTypeGFA' 'Electricity(kWh)'
 'Electricity(kBtu)' 'PrimaryPropertyType']


### Création d'une liste de caractéristiques continues et catégorielles pour différencier quelles caractéristiques nécessitent un traitement.

In [5]:
# Initialisation des listes vides pour les caractéristiques continues et catégorielles
features_continuous = []
features_categorical = []

# Boucle à travers les caractéristiques et identification de leur type de données
for feature in features:
    dtype = data[feature].dtype
    if np.issubdtype(dtype, np.number):  # Vérifiez si c'est un type de données numérique
        features_continuous.append(feature)
        print(dtype)
    else:
        features_categorical.append(feature)

# Affichez les caractéristiques continues et catégorielles
print("Caractéristiques continues :", features_continuous)
print("Caractéristiques catégorielles :", features_categorical)

float64
int64
int64
int64
float64
float64
float64
Caractéristiques continues : ['NumberofBuildings', 'PropertyGFATotal', 'PropertyGFABuilding(s)', 'BuildingAge', 'LargestPropertyUseTypeGFA', 'Electricity(kWh)', 'Electricity(kBtu)']
Caractéristiques catégorielles : ['PrimaryPropertyType']


Les étapes de prétraitement :
Pour les caractéristiques continues, StandardScaler pour les normaliser (centrer et réduire).
Pour les caractéristiques catégorielles, OneHotEncoder pour effectuer le one-hot encoding.

In [6]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), features_continuous),
        ('cat', OneHotEncoder(drop='first'), features_categorical)
    ],
    remainder='passthrough')

# Pipeline complet avec le préprocesseur avec le modèle d'apprentissage supervisé.

In [7]:
print(targets)

['SiteEnergyUse(kBtu)' 'TotalGHGEmissions']


In [8]:
data.columns

Index(['Unnamed: 0', 'OSEBuildingID', 'DataYear', 'BuildingType',
       'PrimaryPropertyType', 'PropertyName', 'Address', 'City', 'State',
       'ZipCode', 'TaxParcelIdentificationNumber', 'CouncilDistrictCode',
       'Neighborhood', 'Latitude', 'Longitude', 'NumberofBuildings',
       'NumberofFloors', 'PropertyGFATotal', 'PropertyGFAParking',
       'PropertyGFABuilding(s)', 'ListOfAllPropertyUseTypes',
       'LargestPropertyUseType', 'LargestPropertyUseTypeGFA',
       'SecondLargestPropertyUseType', 'SecondLargestPropertyUseTypeGFA',
       'ENERGYSTARScore', 'SiteEUI(kBtu/sf)', 'SiteEUIWN(kBtu/sf)',
       'SourceEUI(kBtu/sf)', 'SourceEUIWN(kBtu/sf)', 'SiteEnergyUse(kBtu)',
       'SiteEnergyUseWN(kBtu)', 'SteamUse(kBtu)', 'Electricity(kWh)',
       'Electricity(kBtu)', 'NaturalGas(therms)', 'NaturalGas(kBtu)',
       'DefaultData', 'ComplianceStatus', 'TotalGHGEmissions',
       'GHGEmissionsIntensity', 'BuildingAge', 'ConsommeGaz'],
      dtype='object')

In [9]:
print("Caractéristiques continues :", features_continuous)
print("Caractéristiques catégorielles :", features_categorical)

Caractéristiques continues : ['NumberofBuildings', 'PropertyGFATotal', 'PropertyGFABuilding(s)', 'BuildingAge', 'LargestPropertyUseTypeGFA', 'Electricity(kWh)', 'Electricity(kBtu)']
Caractéristiques catégorielles : ['PrimaryPropertyType']


In [12]:
X = data[features]  # Features 
y = data['SiteEnergyUse(kBtu)']  # Variable cible

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Liste des colonnes numériques
numerical_cols = ['NumberofBuildings', 'PropertyGFATotal', 'PropertyGFABuilding(s)', 'BuildingAge', 'LargestPropertyUseTypeGFA', 'Electricity(kWh)', 'Electricity(kBtu)']

# Liste des colonnes catégorielles (à encoder en one-hot)
categorical_cols = ['PrimaryPropertyType']

# Préprocesseur pour les colonnes numériques
numeric_transformer = StandardScaler()

# Préprocesseur pour les colonnes catégorielles (encodage en one-hot)
categorical_transformer = OneHotEncoder(drop='first')

# Définition du ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Création du pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', LinearRegression())  # Vous pouvez remplacer LinearRegression par un autre modèle
])

# Entraînement du modèle
pipeline.fit(X_train, y_train)



Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['NumberofBuildings',
                                                   'PropertyGFATotal',
                                                   'PropertyGFABuilding(s)',
                                                   'BuildingAge',
                                                   'LargestPropertyUseTypeGFA',
                                                   'Electricity(kWh)',
                                                   'Electricity(kBtu)']),
                                                 ('cat',
                                                  OneHotEncoder(drop='first'),
                                                  ['PrimaryPropertyType'])])),
                ('model', LinearRegression())])

In [13]:
y_pred = pipeline.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

Mean Squared Error: 39824051282878.71
R-squared: 0.9306960908195081


Mean Squared Error Scores: [2.56036579e+14 2.23036480e+13 1.57029213e+14 3.04605850e+13
            nan]
Mean Squared Error Mean: nan
Mean Squared Error Std: nan
NaN dans X_train : 0
NaN dans y_train : 0


/Users/tarek/anaconda3/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:794: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/tarek/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 115, in __call__
    score = scorer._score(cached_call, estimator, *args, **kwargs)
  File "/Users/tarek/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 276, in _score
    y_pred = method_caller(estimator, "predict", X)
  File "/Users/tarek/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 73, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
  File "/Users/tarek/anaconda3/lib/python3.10/site-packages/sklearn/pipeline.py", line 480, in predict
    Xt = transform.transform(Xt)
  File "/Users/tarek/anaconda3/lib/python3.10/site-packages/sklearn/utils/_set_output.py", line 142, in wr